<a href="https://colab.research.google.com/github/Lilchoto3/DS-Unit-2-Applied-Modeling/blob/master/module3-permutation-boosting/LS_DS_233_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf), Chapter 8
  - _**[Gradient Boosting Explained](https://www.gormanalysis.com/blog/gradient-boosting-explained/)**_ — Ben Gorman
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) — Alex Rogozhnikov
  - [How to explain gradient boosting](https://explained.ai/gradient-boosting/) — Terence Parr & Jeremy Howard

In [26]:
!pip install category_encoders==2.*
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
#from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
import category_encoders as ce
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [27]:
# First, I wanna bring in my data and all the changes I made in the last two assignments

train = pd.read_csv('https://raw.githubusercontent.com/Lilchoto3/lilchoto3.github.io/master/Destiny2.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Lilchoto3/lilchoto3.github.io/master/Destiny2_Worthy.csv')

train = train.drop(columns='Unnamed: 0')
test = test.drop(columns='Unnamed: 0')

# swap two sets of perks for Duke Mk. 44, since that weapon in particular is weird.
temp = train.ix[229, 'Perk 2 Rolls']
train.ix[229, 'Perk 2 Rolls'] = train.ix[229, 'Perk 3 Rolls']
train.ix[229, 'Perk 3 Rolls'] = temp
# pd.ix is deprecated, but it works over iloc, so I'm fine with it

# remove some unneeded columns
bad_data = [x for x in train.columns if 'Hash' in x or 'Intrinsic' in x or 'Perk 1' in x or 'Perk 2' in x]
train = train.drop(columns=bad_data)
test = test.drop(columns=bad_data)

# de-string-ify some lists to fiddle with their data
def string_to_list(string_in):
  if isinstance(string_in, str):
    list_out = string_in.replace('[','').replace('"',"'").replace(']','').replace(',','').replace("' '","','").split(',')
    for i, string in enumerate(list_out):
      list_out[i] = string.strip("'")
    return list_out
  else:
    return np.NaN
    
train['Perk 3 Rolls'] = train['Perk 3 Rolls'].apply(string_to_list)
train['Perk 4 Rolls'] = train['Perk 4 Rolls'].apply(string_to_list)
test['Perk 3 Rolls'] = test['Perk 3 Rolls'].apply(string_to_list)
test['Perk 4 Rolls'] = test['Perk 4 Rolls'].apply(string_to_list)

# Get a list of all perks 
perklist = []
for i, perk in enumerate(train[['Perk 3 Init','Perk 3 Rolls','Perk 4 Init','Perk 4 Rolls']].values):
  # Check initial perks first
  if isinstance(perk[0], str):
    if perk[0] not in perklist:
      perklist.append(perk[0])
  if isinstance(perk[2], str):
    if perk[2] not in perklist:
      perklist.append(perk[2])
  # Now check the lists
  if isinstance(perk[1], list):
    for p in perk[1]:
      if p not in perklist:
        perklist.append(p)
  if isinstance(perk[3], list):
    for p in perk[3]:
      if p not in perklist:
        perklist.append(p)
for i, perk in enumerate(test[['Perk 3 Init','Perk 3 Rolls','Perk 4 Init','Perk 4 Rolls']].values):
  # Check initial perks first
  if isinstance(perk[0], str):
    if perk[0] not in perklist:
      perklist.append(perk[0])
  if isinstance(perk[2], str):
    if perk[2] not in perklist:
      perklist.append(perk[2])
  # Now check the lists
  if isinstance(perk[1], list):
    for p in perk[1]:
      if p not in perklist:
        perklist.append(p)
  if isinstance(perk[3], list):
    for p in perk[3]:
      if p not in perklist:
        perklist.append(p)

perklist = pd.Series(perklist)

# Drop some bad perks from year 1 guns or exotics that are only for stats
perklist = perklist.drop([69,70,72,74,83,85,86,92,94,96,101,103,106,108,112,120,139])
perklist = perklist.to_list()

# Encode perks as one-hot columns for each weapon
for perk in perklist:
  train['Rolls '+perk] = 0
  test['Rolls '+perk] = 0

def encode_perks(df):
  for i, weapon in enumerate(df.values):
    # How do I see which rolls the weapons have?
    #print(i,train['Perk 3 Rolls'][i])
    # That works, lets get the initial perks first
    if isinstance(df['Perk 3 Init'][i], str):
      if df['Perk 3 Init'][i] in perklist: # Check for bad perks
        df['Rolls '+df['Perk 3 Init'][i]][i] = 1
        #print(train['Rolls '+train['Perk 3 Init'][i]][i])
    if isinstance(df['Perk 4 Init'][i], str):
      if df['Perk 4 Init'][i] in perklist:
        df['Rolls '+df['Perk 4 Init'][i]][i] = 1
        #print(train['Rolls '+train['Perk 4 Init'][i]][i])
    # Now for the lists
    if isinstance(df['Perk 3 Rolls'][i], list):
      for perk in df['Perk 3 Rolls'][i]:
        if perk in perklist:
          df['Rolls '+perk][i] = 1
          #print(train['Rolls '+perk][i])
    if isinstance(df['Perk 4 Rolls'][i], list):
      for perk in df['Perk 4 Rolls'][i]:
        if perk in perklist:
          df['Rolls '+perk][i] = 1
          #print(train['Rolls '+perk][i])

encode_perks(train)
encode_perks(test)

# drop the columns I just encoded
train = train.drop(columns=['Perk 3 Init','Perk 3 Rolls','Perk 4 Init','Perk 4 Rolls'])
test = test.drop(columns=['Perk 3 Init','Perk 3 Rolls','Perk 4 Init','Perk 4 Rolls'])

# impute missing values
# for weapons from previous seasons, this missing value is the leftover percentage
# of usage divided amongst every weapon without a usage stat
fillna = 100 - train['Usage'].sum()
fillna = fillna / train[train['Usage'].isnull()].shape[0]
fillna

train['Usage'] = train['Usage'].fillna(value=fillna)

# Missing usage values for test data should be zeroes, since the weapons haven't
# been made available to the players yet.
test['Usage'] = test['Usage'].fillna(0)

# Log Transform the Usage target given its skew
train['Usage__Log'] = train['Usage'].apply(np.log)
test['Usage__Log'] = test['Usage'].apply(np.log)

train_seperated = {}
for wep_type in train['Weapon Type'].unique():
  train_seperated[wep_type] = train[train['Weapon Type']==wep_type]
  train_seperated[wep_type] = train_seperated[wep_type].dropna(how='all', axis=1)

# remove charge time and velocity stats from sidearms because
# Devil's Ruin is a cool gun
train_seperated['Sidearm'] = train_seperated['Sidearm'].drop(columns=['Velocity','Charge Time'])

# remove velocity and blast radius for pulse rifles because of bad stats
# also give claws of the wolf a proper impact stat
train_seperated['Pulse Rifle'].ix[132,'Impact'] = 23
train_seperated['Pulse Rifle'] = train_seperated['Pulse Rifle'].drop(columns=['Velocity','Blast Radius'])

# Give 1 second TTKs for Anarchy and Bastion, because they didn't have any before
train_seperated['Grenade Launcher'].ix[464,'Optimal TTK'] = 1
train_seperated['Grenade Launcher'].ix[464,'Bodyshot TTK'] = 1
train_seperated['Fusion Rifle'].ix[465,'Optimal TTK'] = 1
train_seperated['Fusion Rifle'].ix[465,'Bodyshot TTK'] = 1

# Drop Trace Rifles entirely since Trace Rifles are just weird and
# there's not a lot of them and they're all exotic, unique weapons
# and it would be impossible to use them to gauge how good a new one would be.
del train_seperated['Trace Rifle']

test_seperated = {}
for wep_type in test['Weapon Type'].unique():
  test_seperated[wep_type] = test[test['Weapon Type']==wep_type]
  test_seperated[wep_type] = test_seperated[wep_type].dropna(how='all', axis=1)

# Set up the model and model the data
# set up the pipeline
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestRegressor(random_state=42,n_jobs=-1)
)

# set up the variables that will change as it descends
max_depth = {}
n_estimators = {}
max_features = {}
for wep_type in train_seperated:
  max_depth[wep_type] = 20
  n_estimators[wep_type] = 100
  max_features[wep_type] = .3

param_distributions = {}
for wep_type in train_seperated:
  param_distributions[wep_type] = {}

best_models = {}
prev_scores = {}
features = {}

for wep_type in train_seperated:
  for x in [.5, .25, .1]:
    param_distributions[wep_type]['randomforestregressor__max_depth'] = range(
        int(max_depth[wep_type] - (max_depth[wep_type] * x)),
        int(max_depth[wep_type] + (max_depth[wep_type] * x) + 2),
        1
    )
    param_distributions[wep_type]['randomforestregressor__n_estimators'] = range(
        int(n_estimators[wep_type] - (n_estimators[wep_type] * x)),
        int(n_estimators[wep_type] + (n_estimators[wep_type] * x) + 1),
        int(x * 10)
    )
    param_distributions[wep_type]['randomforestregressor__max_features'] = [(x/1000) for x in range(
        int((max_features[wep_type] - (max_features[wep_type] * x)) * 1000),
        int((max_features[wep_type] + (max_features[wep_type] * x)) * 1000 + 1),
        int((max_features[wep_type] / 10) * 1000)
    )]

    # set up the search
    search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_distributions[wep_type],
        n_iter=10,  # 10 times
        cv=4,       # with 4 validation folds
        scoring='neg_mean_absolute_error',  # use MAE to score validation
        verbose=10,
        return_train_score=True,
        n_jobs=-1
    )

    # set up targets and features
    # you need to do this because every weapon has different kinds of stats
    target = 'Usage__Log'
    bad_features = ['Name','Weapon Type','Usage'] + [target]
    features[wep_type] = train_seperated[wep_type].drop(columns=bad_features).columns
    X_train = train_seperated[wep_type][features[wep_type]]
    #print(X_train.head())
    y_train = train_seperated[wep_type][target]
    #print(y_train.head())

    # fit the search
    print(f'Fitting Search for weapon type {wep_type} with hyperparameters:')
    print(f'Max Depth: {max_depth[wep_type]}')
    print(f'Max Depth: {n_estimators[wep_type]}')
    print(f'Max Depth: {max_features[wep_type]}')
    search.fit(X_train, y_train)
    print()

    # Save the best scores and models for later
    if wep_type in best_models:
      # if the score is better, update the best score and the parameters used
      if search.best_score_ > prev_scores[wep_type]:
        prev_scores[wep_type] = search.best_score_
        # update parameters
        max_depth[wep_type] = search.best_params_['randomforestregressor__max_depth']
        n_estimators[wep_type] = search.best_params_['randomforestregressor__n_estimators']
        max_features[wep_type] = search.best_params_['randomforestregressor__max_features']
        # Save the best model by re-creating the pipeline with the best params
        best_models[wep_type] = RandomForestRegressor(
            max_depth=max_depth[wep_type],
            n_estimators=n_estimators[wep_type],
            max_features=max_features[wep_type],
            random_state=42,
            n_jobs=-1
        )
    else:
      # if the scores aren't in the dicts yet, create them
      prev_scores[wep_type] = search.best_score_
      best_models[wep_type] = RandomForestRegressor(
          max_depth=max_depth[wep_type],
          n_estimators=n_estimators[wep_type],
          max_features=max_features[wep_type],
          random_state=42,
          n_jobs=-1
      )
      # update parameters
      max_depth[wep_type] = search.best_params_['randomforestregressor__max_depth']
      n_estimators[wep_type] = search.best_params_['randomforestregressor__n_estimators']
      max_features[wep_type] = search.best_params_['randomforestregressor__max_features']

    # Show me the best so far
    print('Best Hyperparameters:',search.best_params_)
    print('Cross Validation MAE:',search.best_score_)
    print('-----------------------------------------------------------')

Fitting Search for weapon type Scout Rifle with hyperparameters:
Max Depth: 20
Max Depth: 100
Max Depth: 0.3
Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   12.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.



Best Hyperparameters: {'randomforestregressor__n_estimators': 65, 'randomforestregressor__max_features': 0.33, 'randomforestregressor__max_depth': 26}
Cross Validation MAE: -0.4136696957484991
-----------------------------------------------------------
Fitting Search for weapon type Scout Rifle with hyperparameters:
Max Depth: 26
Max Depth: 65
Max Depth: 0.33
Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    8.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.



Best Hyperparameters: {'randomforestregressor__n_estimators': 72, 'randomforestregressor__max_features': 0.412, 'randomforestregressor__max_depth': 20}
Cross Validation MAE: -0.4119379932445064
-----------------------------------------------------------
Fitting Search for weapon type Scout Rifle with hyperparameters:
Max Depth: 20
Max Depth: 72
Max Depth: 0.412
Fitting 4 folds for each of 10 candidates, totalling 40 fits


...

[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    8.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.



Best Hyperparameters: {'randomforestregressor__n_estimators': 71, 'randomforestregressor__max_features': 0.306, 'randomforestregressor__max_depth': 28}
Cross Validation MAE: -0.64647210397287
-----------------------------------------------------------
Fitting Search for weapon type Linear Fusion Rifle with hyperparameters:
Max Depth: 28
Max Depth: 71
Max Depth: 0.306
Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    7.1s



Best Hyperparameters: {'randomforestregressor__n_estimators': 72, 'randomforestregressor__max_features': 0.305, 'randomforestregressor__max_depth': 28}
Cross Validation MAE: -0.6476932179470412
-----------------------------------------------------------


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    7.7s finished


In [31]:
# Use permutation to figure out feature importances

# I wanted to use ELI5 but it doesn't like pipelines, which is sad
from sklearn.model_selection import train_test_split
pd.options.display.max_rows = 150

# pre-encode my categorical variables
encoder = ce.OrdinalEncoder()
X_train_encoded = {}
X_train = {}
y_train = {}
masks = {}
for wep_type in train_seperated:
  X_train[wep_type] = train_seperated[wep_type][features[wep_type]]
  y_train[wep_type] = train_seperated[wep_type][target]
  X_train_encoded[wep_type] = encoder.fit_transform(X_train[wep_type], y_train[wep_type])

# I wanna get the feature importances for each weapon
for wep_type in train_seperated:
  # and I need to do it for every feature for every weapon

  # make a train/val split to get feature importances
  X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
      X_train_encoded[wep_type], y_train[wep_type], test_size=0.2, random_state=42
  )

  best_models[wep_type].fit(X_train_split, y_train_split)

  importances = {}
  for feature in X_train[wep_type].columns:
    # Do the permutation on the validate data
    X_val_permuted = X_val_split.copy()
    X_val_permuted[feature] = np.random.permutation(X_val_split[feature])

    # get the importance of that features
    importances[feature] = best_models[wep_type].score(X_val_split, y_val_split) - best_models[wep_type].score(X_val_permuted, y_val_split)
  
  importances = pd.Series(importances)
  print(f'{wep_type} importances:')
  print(importances.sort_values(ascending=False))

Scout Rifle importances:
Rarity                            1.029867e-01
Archetype                         1.026171e-01
Stability                         6.238009e-02
Rolls Auto-Loading Holster        1.967629e-02
Rolls Zen Moment                  1.620618e-04
Rolls No Distractions             1.110223e-16
Rolls Moving Target               1.110223e-16
Rolls Onslaught                   1.110223e-16
Rolls Multikill Clip              1.110223e-16
Rolls Explosive Light             1.110223e-16
Rolls Serve the Colony            0.000000e+00
Rolls Headseeker                  0.000000e+00
Rolls Slideways                   0.000000e+00
Rolls Feeding Frenzy              0.000000e+00
Rolls Firmly Planted              0.000000e+00
Rolls Disruption Break            0.000000e+00
Rolls Underdog                    0.000000e+00
Rolls Ambitious Assassin          0.000000e+00
Rolls Tap the Trigger             0.000000e+00
Rolls Mulligan                    0.000000e+00
Rolls Tracking Module             0

In [0]:
# Cool, I got all my importances
# I'll implement the masks tomorrow along with whatever else we learn tomorrow